In [41]:
import numpy as np

In [42]:
import pandas as pd

### Алгоритм Нидлмана-Вунша

Глобальное выравнивание

In [56]:
def needleman_wunsch(seq_1, seq_2, match_score, mismatch_score, gap_penalty):
    m = len(seq_1)
    n = len(seq_2)
    F = np.zeros((m+1, n+1))

    # Заполним первую строку и первый столбец штрафами за гэп
    for i in range(m + 1):
        F[i][0] = i * gap_penalty
    for j in range(n + 1):
        F[0][j] = j * gap_penalty

    # Заполним матрицу. Для позиции ij выбираем максимальное значение из (i-1,j), (i, j-1) и (i-1, j-1) позиций
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            match = F[i - 1][j - 1] + (match_score if seq_1[i - 1] == seq_2[j - 1] else mismatch_score)
            delete = F[i - 1][j] + gap_penalty
            insert = F[i][j - 1] + gap_penalty
            F[i][j] = max(match, delete, insert)

    # Пройдемся по таблице в обратном порядке и восстановим оптимальный маршрут, в итоге получим выравнивание.
    align_1, align_2 = "", ""
    i, j = m, n
    while i > 0 or j > 0:
        current_score = F[i][j]
        if i > 0 and j > 0 and current_score == F[i - 1][j - 1] + (match_score if seq_1[i - 1] == seq_2[j - 1] else mismatch_score):
            align_1 += seq_1[i - 1]
            align_2 += seq_2[j - 1]
            i -= 1
            j -= 1
        elif i > 0 and current_score == F[i - 1][j] + gap_penalty:
            align_1 += seq_1[i - 1]
            align_2 += "-"
            i -= 1
        else:
            align_1 += "-"
            align_2 += seq_2[j - 1]
            j -= 1

    return align_1[::-1], align_2[::-1], F[m][n]

#### Пример из таблички:

In [57]:
match_score = 1
mismatch_score = -1
gap_penalty = -1
seq_1 = 'CAGAGTT'
seq_2 = 'CAATTT'

alignment = needleman_wunsch(seq_1, seq_2, match_score, mismatch_score, gap_penalty)
print("Выравнивание:")
print(alignment[0])
print(alignment[1])
print("Score:", alignment[2])

Выравнивание:
CAGAGTT
CA-ATTT
Score: 3.0


Другие примеры:
#### Пример 2

In [45]:
alignment = needleman_wunsch('TACGGGCCCGCTAC', 'TAGCCCTATCGGTCA', 1, -1, -1)
print("Выравнивание:")
print(alignment[0])
print(alignment[1])
print("Score:",alignment[2])

Выравнивание:
TACGGGCC---CGCT-AC
TA--GCCCTATCGGTCA-
Score: 0.0


#### Пример 3
В данной последовательности есть общая часть. Это алгоритм глобального выравнивания, поэтому он попытался их выравнять, но тут много несовпадений, несмотря на попытки добавить гэпы. И скор низкий. Ниже посмотрим, как справится Вотерман-Смит.

In [46]:
alignment = needleman_wunsch('TACCGCCATTTGTCCCGTCGC', 'CGTCGTGCTGCTAACACCATT', 1, -1, -1)
print("Выравнивание:")
print(alignment[0])
print(alignment[1])
print("Score:",alignment[2])

Выравнивание:
TACCGCCAT-TTG-T--C-CCGTCGC
---CGTCGTGCTGCTAACACCAT--T
Score: -4.0


### Алгоритм Смита-Ватермана

Локальное выравнивание

Это код с аффинным штрафом за разрыв, то есть здесь открытие гэпа можно сделать более дорогим, чем его продолжение.

In [47]:
def smith_waterman_aff(seq_1, seq_2, match_score, mismatch_score, gap_open_penalty, gap_extend_penalty):
    m = len(seq_1)
    n = len(seq_2)

    # Создаем 3 матрицы
    H = np.zeros((m + 1, n + 1))  # Основная матрица
    E = np.zeros((m + 1, n + 1))  # Матрица для гэпов в seq_1
    F = np.zeros((m + 1, n + 1))  # Матрица для гэпов в seq_2

    max_score = 0
    max_pos = (0, 0)

    # Заполняем матрицы. 
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            match = H[i - 1][j - 1] + (match_score if seq_1[i - 1] == seq_2[j - 1] else mismatch_score)
            F[i][j] = max(F[i - 1][j] + gap_extend_penalty, H[i - 1][j] + gap_open_penalty)  
            E[i][j] = max(E[i][j - 1] + gap_extend_penalty, H[i][j - 1] + gap_open_penalty)  
            H[i][j] = max(0, match, E[i][j], F[i][j]) 

            if H[i][j] > max_score:
                max_score = H[i][j]
                max_pos = (i, j)

    # Восстановление выравнивания
    align_1, align_2 = "", ""
    i, j = max_pos

    # Определяем, как пришли в текущую ячейку. Здесь важно учесть, что если мы пришли в Н из матрицы Е или F, то там может сидеть продолжение гэпа. До того
    #как внутри elif'ов я добавила while, он работал некорректно и делал дополнительный гэп. 
    while H[i][j] > 0:
        if H[i][j] == H[i - 1][j - 1] + (match_score if seq_1[i - 1] == seq_2[j - 1] else mismatch_score):
            # Пришли по диагонали (совпадение или несовпадение)
            align_1 += seq_1[i - 1]
            align_2 += seq_2[j - 1]
            i -= 1
            j -= 1
        elif H[i][j] == E[i][j]:
            # Пришли из E (гэп в seq_1)
            align_1 += "-"
            align_2 += seq_2[j - 1]
            j -= 1
            # Продолжаем идти по E, если это продолжение гэпа
            while j > 0 and H[i][j] == E[i][j]:
                align_1 += "-"
                align_2 += seq_2[j - 1]
                j -= 1
        elif H[i][j] == F[i][j]:
            # Пришли из F (гэп в seq_2)
            align_1 += seq_1[i - 1]
            align_2 += "-"
            i -= 1
            # Продолжаем идти по F, если это продолжение гэпа
            while i > 0 and H[i][j] == F[i][j]:
                align_1 += seq_1[i - 1]
                align_2 += "-"
                i -= 1

    return align_1[::-1], align_2[::-1], H[m][n]

#### Пример из таблички:

In [61]:
alignment = smith_waterman_aff('CAGAGTT', 'CAATTT', 1, -1, -1, -1)
print("Выравнивание:")
print(alignment[0])
print(alignment[1])
print("Score:", alignment[2])

Выравнивание:
CAGAGTT
CA-A-TT
Score: 3.0


Другие примеры:
#### Пример 2

In [49]:
alignment = smith_waterman_aff('TACGGGCCCGCTAC', 'TAGCCCTATCGGTCA', 5, -4, -5, -1)
print("Выравнивание:")
print(alignment[0])
print(alignment[1])
print("Score:",alignment[2])

Выравнивание:
TACGGGCCCGCTA
TA---GCC--CTA
Score: 18.0


#### Пример 3
С таким примером локальное выравнивание справилось лучше, чем глобальное, и нашло общую часть последовательности (с двумя мисмэтчами):

In [50]:
alignment = smith_waterman_aff('TACCGCCATTTGTCCCGTCGC', 'CGTCGTGCTGCTAACACCATT', 5, -4, -5, -1)
print("Выравнивание:")
print(alignment[0])
print(alignment[1])
print("Score:",alignment[2])

Выравнивание:
TACCGCCATT
TAACACCATT
Score: 17.0


### Реализованные Python-пакеты

In [51]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

Здесь читала про возможности модуля: https://biopython.org/docs/1.76/api/Bio.pairwise2.html

#### Пример из таблички

In [52]:
seq_1 = 'CAGAGTT'
seq_2 = 'CAATTT'

alignments = pairwise2.align.globalms(seq_1, seq_2, 1, -1, -1, -1)

for alignment in alignments:
    print(format_alignment(*alignment))

CAGAGTT
|| |.||
CA-ATTT
  Score=3



Совпадает с тем, что выдал мой код алгоритма Нидлмана-Вунша. Если не устанавливать собственных парамтеров, то можно использовать функцию globalxx, у нее match = 1 и не присваивает никакого наказания для mismatch и gap. Она выдает больше вариантов выравнивания.

#### Пример 2

In [53]:
seq_1 = 'TACGGGCCCGCTAC' 
seq_2 = 'TAGCCCTATCGGTCA'


alignments = pairwise2.align.globalms(seq_1, seq_2, 1, -1, -1, -1)

for alignment in alignments:
    print(format_alignment(*alignment))


TACGGGCC---CGCTAC-
||  |.||   ||.| | 
TA--GCCCTATCGGT-CA
  Score=0

TACGGGCC---CGCTAC-
|| | .||   ||.| | 
TA-G-CCCTATCGGT-CA
  Score=0

TACGGGCC---CGCTAC-
|| |. ||   ||.| | 
TA-GC-CCTATCGGT-CA
  Score=0

TACGGGCC---CGCT-AC
||  |.||   ||.| | 
TA--GCCCTATCGGTCA-
  Score=0

TACGGGCC---CGCT-AC
|| | .||   ||.| | 
TA-G-CCCTATCGGTCA-
  Score=0

TACGGGCC---CGCT-AC
|| |. ||   ||.| | 
TA-GC-CCTATCGGTCA-
  Score=0



Первое выравнивание совпадает с полученным с помощью алгоритма Нидлмана Вунша.

#### Пример 3

In [54]:
seq_1 = 'TACCGCCATTTGTCCCGTCGC'
seq_2 = 'CGTCGTGCTGCTAACACCATT'


alignments = pairwise2.align.globalms(seq_1, seq_2, 1, -1, -1, -1)

for alignment in alignments:
    print(format_alignment(*alignment))


TACCGCCATT-TG-T--C-CCGTCGC
   ||.|.|. || |  | ||.|  .
---CGTCGTGCTGCTAACACCAT--T
  Score=-4

TACCGCCATT-TG-T--C-CCGTCGC
  | |.|.|. || |  | ||.|  .
--C-GTCGTGCTGCTAACACCAT--T
  Score=-4

TACCGCCAT-TTG-T--C-CCGTCGC
   ||.|.| .|| |  | ||.|  .
---CGTCGTGCTGCTAACACCAT--T
  Score=-4

TACCGCCAT-TTG-T--C-CCGTCGC
  | |.|.| .|| |  | ||.|  .
--C-GTCGTGCTGCTAACACCAT--T
  Score=-4

TACCGCCATT-TG-T--C-CCGTCGC
   ||.|.|. || |  | ||.| . 
---CGTCGTGCTGCTAACACCAT-T-
  Score=-4

TACCGCCATT-TG-T--C-CCGTCGC
  | |.|.|. || |  | ||.| . 
--C-GTCGTGCTGCTAACACCAT-T-
  Score=-4

TACCGCCAT-TTG-T--C-CCGTCGC
   ||.|.| .|| |  | ||.| . 
---CGTCGTGCTGCTAACACCAT-T-
  Score=-4

TACCGCCAT-TTG-T--C-CCGTCGC
  | |.|.| .|| |  | ||.| . 
--C-GTCGTGCTGCTAACACCAT-T-
  Score=-4

TACCGCCATT-TG-T--C-CCGTCGC
   ||.|.|. || |  | ||.|.  
---CGTCGTGCTGCTAACACCATT--
  Score=-4

TACCGCCATT-TG-T--C-CCGTCGC
  | |.|.|. || |  | ||.|.  
--C-GTCGTGCTGCTAACACCATT--
  Score=-4

TACCGCCAT-TTG-T--C-CCGTCGC
   ||.|.| .|| |  | ||.|.  
---CGTCGTGCTGCTA

Третье выравнивание совпадает с моим.